In [1]:
# Grammar dalam bentuk CNF untuk CYK Parser
grammar = {
    'S': [['NP', 'VP']],
    'VP': [['V', 'NP'], ['eats']],
    'NP': [['Det', 'N'], ['she']],
    'Det': [['a']],
    'N': [['apple']],
    'V': [['eats']]
}

# Fungsi untuk mengecek CYK Parsing
def cyk_parser(grammar, string):
    words = string.split()
    n = len(words)
    table = [[set() for _ in range(n)] for _ in range(n)]
    
    # Inisialisasi tabel parsing
    for i in range(n):
        for lhs, rhs in grammar.items():
            for rule in rhs:
                if len(rule) == 1 and rule[0] == words[i]:
                    table[i][i].add(lhs)
    
    # Isi tabel untuk substring lebih panjang
    for l in range(2, n + 1):
        for i in range(n - l + 1):
            j = i + l - 1
            for k in range(i, j):
                for lhs, rhs in grammar.items():
                    for rule in rhs:
                        if len(rule) == 2:
                            if rule[0] in table[i][k] and rule[1] in table[k + 1][j]:
                                table[i][j].add(lhs)
    
    return 'S' in table[0][n - 1]

# Uji parser dengan string
print(cyk_parser(grammar, "she eats a apple"))  # Output: True jika string sesuai dengan grammar

True


In [2]:
# Grammar untuk Earley Parser
grammar = {
    'S': [['NP', 'VP']],
    'VP': [['V', 'NP'], ['eats']],
    'NP': [['Det', 'N'], ['she']],
    'Det': [['a']],
    'N': [['apple']],
    'V': [['eats']]
}

# Kelas untuk State dalam tabel parsing
class State:
    def __init__(self, lhs, rhs, dot, start):
        self.lhs = lhs
        self.rhs = rhs
        self.dot = dot
        self.start = start

    def is_complete(self):
        return self.dot >= len(self.rhs)

    def next_symbol(self):
        if self.dot < len(self.rhs):
            return self.rhs[self.dot]
        return None

    def __repr__(self):
        return f"{self.lhs} -> {self.rhs[:self.dot]} • {self.rhs[self.dot:]} (start={self.start})"


# Fungsi Earley Parsing dengan operasi predict, scan, dan complete
def earley_parser(grammar, word):
    chart = [[] for _ in range(len(word) + 1)]
    start_state = State('S', grammar['S'][0], 0, 0)
    chart[0].append(start_state)

    for i in range(len(word) + 1):
        for state in chart[i]:
            if state.is_complete():
                complete(state, chart, i)
            elif state.next_symbol() in grammar:
                predict(state, grammar, chart, i)
            else:
                scan(state, word, chart, i)

    for state in chart[len(word)]:
        if state.lhs == 'S' and state.is_complete() and state.start == 0:
            return True, chart
    return False, chart

def predict(state, grammar, chart, position):
    symbol = state.next_symbol()
    if symbol in grammar:
        for production in grammar[symbol]:
            new_state = State(symbol, production, 0, position)
            add_state(new_state, chart, position)

def scan(state, word, chart, position):
    symbol = state.next_symbol()
    if position < len(word) and symbol == word[position]:
        new_state = State(state.lhs, state.rhs, state.dot + 1, state.start)
        add_state(new_state, chart, position + 1)

def complete(state, chart, position):
    for s in chart[state.start]:
        if not s.is_complete() and s.next_symbol() == state.lhs:
            new_state = State(s.lhs, s.rhs, s.dot + 1, s.start)
            add_state(new_state, chart, position)

def add_state(state, chart, position):
    if state not in chart[position]:
        chart[position].append(state)
        return True
    return False

# Uji parser dengan string
input_string = "she eats a apple".split()
is_parsed, parsing_chart = earley_parser(grammar, input_string)

# Output hasil parsing
if is_parsed:
    print("String diterima oleh grammar.")
else:
    print("String tidak diterima oleh grammar.")

print("\nTabel Parsing:")
for i, states in enumerate(parsing_chart):
    print(f"Posisi {i}:")
    for state in states:
        print("   ", state)

String diterima oleh grammar.

Tabel Parsing:
Posisi 0:
    S -> [] • ['NP', 'VP'] (start=0)
    NP -> [] • ['Det', 'N'] (start=0)
    NP -> [] • ['she'] (start=0)
    Det -> [] • ['a'] (start=0)
Posisi 1:
    NP -> ['she'] • [] (start=0)
    S -> ['NP'] • ['VP'] (start=0)
    VP -> [] • ['V', 'NP'] (start=1)
    VP -> [] • ['eats'] (start=1)
    V -> [] • ['eats'] (start=1)
Posisi 2:
    VP -> ['eats'] • [] (start=1)
    V -> ['eats'] • [] (start=1)
    S -> ['NP', 'VP'] • [] (start=0)
    VP -> ['V'] • ['NP'] (start=1)
    NP -> [] • ['Det', 'N'] (start=2)
    NP -> [] • ['she'] (start=2)
    Det -> [] • ['a'] (start=2)
Posisi 3:
    Det -> ['a'] • [] (start=2)
    NP -> ['Det'] • ['N'] (start=2)
    N -> [] • ['apple'] (start=3)
Posisi 4:
    N -> ['apple'] • [] (start=3)
    NP -> ['Det', 'N'] • [] (start=2)
    VP -> ['V', 'NP'] • [] (start=1)
    S -> ['NP', 'VP'] • [] (start=0)
